In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from tabulate import tabulate
import statistics as stat
import pandas as pd
import numpy as np
import requests
import math
import json

In [2]:
# Задание 2.1
# 0. Чтение данных из подготовленных файлов и объявление констант.
data = pd.read_csv("../resources/data.csv", delimiter=', ').drop(columns=["Unnamed: 0"])
context_day = pd.read_csv("../resources/context_day.csv", delimiter=', ').drop(columns=["Unnamed: 0"])
context_place = pd.read_csv("../resources/context_place.csv", delimiter=', ').drop(columns=["Unnamed: 0"])

my_index = 11
non_rated = -1

home = 'h'
non_home = ['v', 'c']

weekends = ['Sat', 'Sun']
non_weekends = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']

def r3(x):
    return round(x, 3)

# 1. Для заданного пользователя (12 Ковшов Валерий) рассчитать оценки для всех фильмов, которые он не оценил.
my_data_row = data.iloc[[my_index]].values[0]
row_len = len(my_data_row)

sims = {}
for row_index in range(len(data)):
    if row_index == my_index:
        continue
    user_data_row = data.iloc[[row_index]].values[0]
    sum_ui_vi = 0
    sum_ui = 0
    sum_vi = 0
    for column_index in range(row_len):
        my_data_cell = my_data_row[column_index]
        user_data_cell = user_data_row[column_index]
        if (my_data_cell != non_rated) & (user_data_cell != non_rated):
            sum_ui_vi += my_data_cell * user_data_cell
            sum_ui += my_data_cell ** 2
            sum_vi += user_data_cell ** 2
    sim_u_v = r3(sum_ui_vi / (math.sqrt(sum_ui) * math.sqrt(sum_vi)))
    sims[row_index] = sim_u_v

sorted_sims = dict(sorted(sims.items(), key=lambda x: x[1], reverse=True))

avg_for_user = {}
for row_index in range(len(data)):
    avg = 0
    count = 0
    data_row = data.iloc[[row_index]].values[0]
    for column_index in range(row_len):
        data_cell = data_row[column_index]
        if data_cell != non_rated:
            avg += data_cell
            count += 1
    avg_for_user[row_index] = r3(avg / count)

my_non_rated = []
future_rate = {}
for movie_index in range(row_len):
    if my_data_row[movie_index] == non_rated:
        my_non_rated.append(movie_index)
        sum_numerator = 0
        sum_denominator = 0
        count = 0
        for user_index in sorted_sims:
            user_rate = data.iloc[[user_index]].values[0][movie_index]
            if user_rate == non_rated:
                count += 1
            elif count == 4:
                break
            else:
                count += 1
                sum_numerator += sims[user_index] * (user_rate - avg_for_user[user_index]) 
                sum_denominator += sims[user_index]
        future_rate[data.columns[movie_index]] = r3(avg_for_user[my_index] + sum_numerator / sum_denominator)

# 2. Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных.
# Мы будем выбирать из фильмов, которые пользователь ещё не смотрел иначе какой смысл? Ориенироваться при выборе будем на оценки пользователей, которые смотрели эти фильмы (которые не видел заданный пользователь) дома в выходные с учётом соотношения вкусов этих пользователей с вкусами заданного пользователя.
avg_for_movie = {}
for column_index in range(len(my_non_rated)):
    avg = 0
    count = 0
    data_column = data[data.columns[my_non_rated[column_index]]]
    for row_index in range(len(data)):
        data_cell = data_column[row_index]
        if data_cell != non_rated:
            avg += data_cell
            count += 1
    avg_for_movie[my_non_rated[column_index]] = r3(avg / count)

context_day_for_non_rated = context_day[context_day.columns[my_non_rated]]
context_day_for_non_rated_at_weekends = context_day_for_non_rated.replace(weekends, 1).replace(non_weekends, 0)

context_place_for_non_rated = context_place[context_place.columns[my_non_rated]]
context_place_for_non_rated_at_home = context_place_for_non_rated.replace(home, 1).replace(non_home, 0)

weekends_at_home = {}
for user_index in range(len(data)):
    weekends_at_home[user_index] = []
    user_context_day = context_day_for_non_rated_at_weekends.iloc[[user_index]].values[0]
    user_context_place = context_place_for_non_rated_at_home.iloc[[user_index]].values[0]
    for movie_index in range(len(my_non_rated)):
        context_day_cell = int(user_context_day[movie_index])
        context_place_cell = int(user_context_place[movie_index])
        if context_day_cell == 1 & context_place_cell == 1:
            weekends_at_home[user_index].append(True)
        else:
            weekends_at_home[user_index].append(False)

rate_for_non_rated_at_weekends_at_home = {}
for movie_index in range(len(my_non_rated)):
    sum_numerator = 0
    sum_denominator = 0
    for user_index in weekends_at_home:
        if weekends_at_home[user_index][movie_index] == False:
            continue
        sum_numerator += sims[user_index] * (data.iloc[[user_index]].values[0][my_non_rated[movie_index]] - avg_for_movie[my_non_rated[movie_index]])
        sum_denominator += sims[user_index]
    rate_for_non_rated_at_weekends_at_home[data.columns[my_non_rated[movie_index]]] = avg_for_movie[my_non_rated[movie_index]] + r3(sum_numerator / sum_denominator)
top_movie_for_weekends_at_home_tuple = sorted(rate_for_non_rated_at_weekends_at_home.items(), key=lambda x: x[1], reverse=True)[0]
top_movie_for_weekends_at_home = {top_movie_for_weekends_at_home_tuple[0]: top_movie_for_weekends_at_home_tuple[1]}

# 3. Заполнение json файла.
user = my_index + 1
ans_part1 = json.dumps(future_rate, sort_keys=True, indent=4)
ans_part2 = json.dumps(top_movie_for_weekends_at_home, sort_keys=True, indent=4) 
result = json.dumps({'user': user,'1': future_rate, '2': top_movie_for_weekends_at_home}, indent=4)
print(result)

{
    "user": 12,
    "1": {
        "Movie 4": 3.538,
        "Movie 6": 3.92,
        "Movie 11": 2.556,
        "Movie 20": 3.591
    },
    "2": {
        "Movie 4": 3.199
    }
}


In [3]:
# Задание 3.2
movie_names = pd.read_csv("../resources/movie_names.csv", delimiter=', ')
movie_to_name = dict(movie_names.values[[i - 1 for i in my_non_rated]])
movie_to_name

{'Movie 4': 'The Dark Knight',
 'Movie 6': "Schindler's List",
 'Movie 11': 'The Lord of the Rings: The Fellowship of the Ring',
 'Movie 20': 'City of God'}

In [4]:
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

movie_to_id = {}
for movie in movie_to_name:
    expected_label = movie_to_name[movie]
    params = {
        'action' : 'wbsearchentities',
        'format' : 'json',
        'language' : 'en',
        'search': expected_label
    }
    res = requests.get(API_ENDPOINT, params = params)
    json_search = res.json()['search']
    film = {}
    for i in range(len(json_search)):
        if "film" in json_search[i]['description']:
            film = json_search[i]
            break
    print(
        "\nExpected title:\t", expected_label,
        "\nActual title:\t", film['label'],
        "\nDescription:\t", film['description'],
        "\nId:\t\t", film['id']
    )
    movie_to_id[movie] = film['id']
movie_to_id


Expected title:	 The Dark Knight 
Actual title:	 The Dark Knight 
Description:	 2008 British-American superhero film directed by Christopher Nolan 
Id:		 Q163872

Expected title:	 Schindler's List 
Actual title:	 Schindler's List 
Description:	 1993 film by Steven Spielberg 
Id:		 Q483941

Expected title:	 The Lord of the Rings: The Fellowship of the Ring 
Actual title:	 The Lord of the Rings: The Fellowship of the Ring 
Description:	 2001 film directed by Peter Jackson 
Id:		 Q127367

Expected title:	 City of God 
Actual title:	 City of God 
Description:	 2002 film directed by Fernando Meirelles and Kátia Lund 
Id:		 Q220741


{'Movie 4': 'Q163872',
 'Movie 6': 'Q483941',
 'Movie 11': 'Q127367',
 'Movie 20': 'Q220741'}

In [5]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

movie_to_results_df = {}
for movie in movie_to_id:
    id = movie_to_id[movie]
    sparql_query = """

SELECT
  ?filmLabel
  ?yearLabel
  ?cannesAwardLabel
WHERE {
  { SELECT (MIN(YEAR(?publicationDate)) AS ?year) WHERE { wd:""" + id + """ wdt:P577 ?publicationDate. } }
  ?cannesAward wdt:P31 wd:Q28444913.
  ?awardStatement ps:P166 ?cannesAward;
                  pq:P585 ?awardDate.
  ?film wdt:P31 wd:Q11424;
        p:P166 ?awardStatement.
  FILTER((YEAR(?awardDate)) = ?year )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
 
"""
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    movie_to_results_df[movie] = results_df[['filmLabel.value', 'yearLabel.value', 'cannesAwardLabel.value']].head()

for movie in movie_to_results_df:
    print(
        movie, "\n",
        tabulate(movie_to_results_df[movie], headers='keys', tablefmt='psql'), "\n"
    )

Movie 4 
 +----+-------------------+-------------------+---------------------------------+
|    | filmLabel.value   |   yearLabel.value | cannesAwardLabel.value          |
|----+-------------------+-------------------+---------------------------------|
|  0 | Gomorrah          |              2008 | Cannes Film Festival Grand Prix |
|  1 | Hunger            |              2008 | Caméra d'Or                     |
|  2 | The Class         |              2008 | Palme d'Or                      |
|  3 | Il Divo           |              2008 | Jury Prize                      |
+----+-------------------+-------------------+---------------------------------+ 

Movie 6 
 +----+-----------------------+-------------------+---------------------------------------------+
|    | filmLabel.value       |   yearLabel.value | cannesAwardLabel.value                      |
|----+-----------------------+-------------------+---------------------------------------------|
|  0 | The Piano             |         